# Query

In [ ]:
from py2neo import Graph
from py2neo.data import Node, Relationship
from pprint import pprint

### Connessione al DB

In [ ]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "pokemon"))

### Esequzione delle query

Quali sono le varianti del Pokemon Charizard?

In [ ]:
query1 = '''
MATCH (p:Pokemon) -[r:HAS_VARIANT]->(v:Pokemon)
WHERE p.Name='Charizard'
RETURN p.Name AS PokemonNonVariante, v.Name AS Variante
'''
# Oppure
# Runnare su Neo4j
query2= '''
MATCH (p:Pokemon) -[r:HAS_VARIANT]->(v:Pokemon)
WHERE p.Name='Charizard'
RETURN p,r,v
'''

In [ ]:
graph.run(query1)
cursor = graph.run(query1)
print(cursor.to_table())

 PokemonNonVariante | Variante             
--------------------|----------------------
 Charizard          | Charizard Gigantamax 
 Charizard          | Mega Charizard Y     
 Charizard          | Mega Charizard X     



Quali sono i compagni più frequenti del Pokemon Charizard? (ho considerato come soglia il 30%)

In [ ]:
query3='''
 MATCH (p:Pokemon)-[r:USED_IN_TEAM_WITH]->(t:Pokemon)
WHERE p.Name='Charizard'AND r.UsePercentage > 30
RETURN t.Name AS TeamMate, r.UsePercentage AS Percentuale
ORDER BY r.UsePercentage  DESC
'''
#Possiamo anche stampare il pokemon più utilizzato:
query4='''
MATCH (p:Pokemon {Name: 'Charizard'})-[r:USED_IN_TEAM_WITH]->(t:Pokemon)
WITH max(r.UsePercentage) AS max
MATCH (p:Pokemon)-[r:USED_IN_TEAM_WITH]->(t:Pokemon)
WHERE r.UsePercentage = max
RETURN p.Name AS Pokemon ,t.Name AS TeamMate, r.UsePercentage AS Percentuale


'''

In [ ]:
graph.run(query3)
cursor = graph.run(query3)
print(cursor.to_table())

 TeamMate             | Percentuale 
----------------------|-------------
 Groudon              |      86.831 
 Incineroar           |      70.405 
 Zacian Crowned Sword |      58.274 
 Grimmsnarl           |      57.381 
 Gastrodon            |      47.798 



In [ ]:
graph.run(query4)
cursor = graph.run(query4)
print(cursor.to_table())

 Pokemon   | TeamMate | Percentuale 
-----------|----------|-------------
 Charizard | Groudon  |      86.831 



Quali sono le mosse che possono essere imparate dal Pokemon Charizard?

In [ ]:
query5='''
MATCH (m:Moves)<-[r:MAY_LEARN]-(p:Pokemon)
WHERE p.Name='Charizard'
RETURN m.Name AS Mossa, p.Name AS POkemon
'''

In [ ]:
graph.run(query5)
cursor = graph.run(query5)
print(cursor.to_table())

 Mossa           | POkemon   
-----------------|-----------
 Earthquake      | Charizard 
 Hone Claws      | Charizard 
 Mystical Fire   | Charizard 
 Crunch          | Charizard 
 Scary Face      | Charizard 
 Facade          | Charizard 
 Fire Fang       | Charizard 
 Aerial Ace      | Charizard 
 Rock Smash      | Charizard 
 Cut             | Charizard 
 Frustration     | Charizard 
 Twister         | Charizard 
 Mega Punch      | Charizard 
 Endure          | Charizard 
 Return          | Charizard 
 Dragon Claw     | Charizard 
 Steel Wing      | Charizard 
 Beat Up         | Charizard 
 Focus Blast     | Charizard 
 Scorching Sands | Charizard 
 Scale Shot      | Charizard 
 Incinerate      | Charizard 
 Fire Pledge     | Charizard 
 Fire Blast      | Charizard 
 Attract         | Charizard 
 Fly             | Charizard 
 Work Up         | Charizard 
 Rock Slide      | Charizard 
 Smokescreen     | Charizard 
 Fire Punch      | Charizard 
 Fire Spin       | Charizard 
 Blast Bur

Quali, tra le mosse più utilizzate con il pokemon Charizard, sono superefficaci sul pokemon Arcanine?

In [ ]:
#Runnare su Neo4j

# Pokemon y
query6='''
MATCH (p:Pokemon{Name:'Arcanine'})-[rt:IS_OF_TYPE]-> (t:Type)
RETURN p,rt,t
'''

# Pokemon x AND query (bisogna sistemare il RETURN):

query7='''
MATCH (p:Pokemon {Name:'Charizard'})-[r:USED_WITH_MOVE]->(m:Moves) 
WHERE r.UsePercentage >30
MATCH (p)-[rt:IS_OF_TYPE]-> (tp:Type)
MATCH (m)-[rm:MOVES_IS_TYPE]->(tm)
CALL {
    WITH tm
    MATCH (t)-[eff:MOVE_EFFECTIVENESS_ON_POKEMON]->(teff:Type)
    WHERE teff.Name='fire' AND eff.Effectiveness='Super-effective (200%)'
    RETURN eff, teff
}
RETURN * 
'''


Quali Pokemon di tipo fire vengono usati con l’abilità Solar Power?

In [ ]:
query8='''
MATCH (p:Pokemon)-[r:IS_OF_TYPE]-> (t:Type {Name:'fire'})
CALL {
    WITH p
    MATCH (p)-[u:USED_WITH_ABILITY]->(a:Abilities {Name:'Solar Power'})
    RETURN u,a
}
RETURN p.Name AS Pokemon,a.Name AS Abilità, t.Name AS Tipo
'''

In [ ]:
graph.run(query8)
cursor = graph.run(query8)
print(cursor.to_table())

 Pokemon       | Abilità     | Tipo 
---------------|-------------|------
 Mega Houndoom | Solar Power | fire 
 Charmander    | Solar Power | fire 
 Charmeleon    | Solar Power | fire 
 Charizard     | Solar Power | fire 



Quali sono i Pokemon più utilizzati nel VGC? (soglia 20%)

In [ ]:
query9='''
MATCH (p:Pokemon)
WHERE p.UsagePercentMarch2022 > 20
RETURN p.Name AS Pokemon, p.UsagePercentMarch2022 AS Percentuale
ORDER BY p.UsagePercentMarch2022 DESC
'''

In [ ]:
graph.run(query9)
cursor = graph.run(query9)
print(cursor.to_table())

 Pokemon                   | Percentuale 
---------------------------|-------------
 Zacian Crowned Sword      |          65 
 Incineroar                |          59 
 Kyogre                    |          35 
 Grimmsnarl                |          30 
 Regieleki                 |          29 
 Rillaboom                 |          27 
 Groudon                   |          23 
 Calyrex Shadow Rider      |          22 
 Thundurus Incarnate Forme |          21 

